In [41]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%autoreload 2
import torch
import pandas as pd 
from ultralytics import YOLO
import numpy as np
import supervision as sv
import logging
from tkinter import Tk,filedialog
import os 
from pathlib import Path
import csv
import cv2
from typing import Optional,Sequence, Tuple
import warnings
from collections import deque
from ball_tracker import BallTracker
from ball_trajectory import BallTrajectoryAnnotator
from bounce_annotator import BounceAnnotator
from bounce_detector import BounceDetector,ConfirmBounceDetector,AngleBounceDetector

In [43]:
model = YOLO('../Models/2025_v0.pt')
#source_file =  add code for file diaglouge to open to laod video 
#target_file = get the file name freom sourcepath and add_processed before the file extgension. 
logging.basicConfig(level=logging.INFO)
logging.info('Started Inference')

root=Tk()
root.withdraw()
source_file = filedialog.askopenfilename(title="Select source video file",filetypes=[("Video files", "*.mp4;*.avi;*.mov"), ("All files", "*.*")])
if not source_file:
    logging.error("No source file selected. Exiting.")
    exit()
directory,filename = os.path.split(source_file)
name,ext = os.path.splitext(filename)
target_file_name = name + '_processed' + ext
target_file = os.path.join(directory,target_file_name)

INFO:root:Started Inference


In [44]:
def run_inference(
    model,
    frame: np.ndarray,
    frame_number: int,
    *,
    smoother: BallTracker,
    trajectory_annotator: BallTrajectoryAnnotator,
    angle_detector: AngleBounceDetector,
    bounce_annotator: BounceAnnotator,
    save_csv: bool = False,
    writer=None,
    conf_threshold: float = 0.25,
    iou_threshold: float = 0.45,
    court =None
) -> np.ndarray:
    if save_csv and writer is None:
        raise ValueError("writer required if save_csv=True")

    # 1) YOLO detection
    res  = model(frame, conf=conf_threshold, iou=iou_threshold)[0]
    dets = sv.Detections.from_ultralytics(res)

    # 2) smoothing & selection
    chosen_idx, chosen_box, chosen_center = smoother.update(dets.xyxy)

    # 3) prepare for annotation
    if chosen_idx is not None:
        det_to_draw = sv.Detections(
            xyxy       = np.array([chosen_box]),
            confidence = np.array([dets.confidence[chosen_idx]]),
            class_id   = np.array([dets.class_id[chosen_idx]])
        )
    else:
        det_to_draw = dets

    # 4) draw boxes
    img = sv.BoxAnnotator().annotate(frame, det_to_draw)
    if court:
        img = cv2.polylines(img,court,True,(0,0,100),2)
        print('Added court')
    
    # 5) overlay frame number
    cv2.putText(
        img, str(frame_number),
        (10,30), cv2.FONT_HERSHEY_COMPLEX,
        1, (0,255,0), 2, cv2.LINE_AA
    )

    # 6) draw trajectory
    traj_str = ""
    if smoother.trajectory:
        img = trajectory_annotator.add_trajectory(img, smoother.trajectory)
        traj_str = str(list(smoother.trajectory))

    # 7) angle-based bounce detection
    traj = list(smoother.trajectory)
    bounced, angle_deg = angle_detector.detect(frame_number, traj)

    # choose center for annotation (bounce happens @ previous frame)
    if bounced and len(smoother.trajectory) >= 2:
        bounce_center = smoother.trajectory[-2]
    else:
        bounce_center = chosen_center

    img = bounce_annotator.annotate(
        img,
        bounced=bounced,
        center=bounce_center,
        frame_number=frame_number
    )

    # 8) CSV logging
    if save_csv:
        n_fields = 18
        if det_to_draw.xyxy.size == 0:
            writer.writerow([frame_number] + [""] * n_fields)
        else:
            for det_idx, (xy, conf) in enumerate(zip(det_to_draw.xyxy, det_to_draw.confidence), start=1):
                x1, y1, x2, y2 = map(float, xy)
                c_x, c_y      = (x1 + x2) / 2, (y1 + y2) / 2

                if chosen_idx is not None:
                    cx1, cy1, cx2, cy2 = map(float, chosen_box)
                    ccx, ccy = chosen_center
                    chosen_id = chosen_idx + 1
                else:
                    cx1 = cy1 = cx2 = cy2 = ccx = ccy = ""
                    chosen_id = ""

                angle_str = f"{angle_deg:.1f}" if angle_deg is not None else ""
                prev_bounce = int(bounced)

                writer.writerow([
                    frame_number,
                    det_idx,
                    conf,
                    c_x, c_y,
                    x1, y1, x2, y2,
                    chosen_id,
                    cx1, cy1, cx2, cy2,
                    ccx, ccy,
                    traj_str,
                    angle_str,
                    prev_bounce
                ])

    return img


try:
    csv_file = '../Data/detections.csv'
    tracker = BallTracker(
        max_history=10,
        static_thresh=5,
        stable_frames=5,
        lock_frames=3,
        max_age=5)
    traj_annot = BallTrajectoryAnnotator(color=(0,255,0), thickness=2)
    angle_detector = AngleBounceDetector(min_angle_deg=40, debug=True)
    bounce_annotator = BounceAnnotator()
    f = open(csv_file, mode='w', newline='')
    writer = csv.writer(f)
    writer.writerow([
        "frame_number",
        "det_idx",
        "confidence",
        "c_x","c_y",
        "x1","y1","x2","y2",
        "chosen_id",
        "chosen_x1","chosen_y1","chosen_x2","chosen_y2",
        "chosen_cx","chosen_cy",
        "trajectory",
        "angle",
        "prev_frame_bounce"
    ])
    court_coordinates =   [np.array([[506, 68], [798, 64], [1167, 564], [207, 569]])]
    sv.process_video(
        source_path=source_file,
        target_path=target_file,
        callback=lambda frame, fn: run_inference(
            model, frame, fn,
            smoother=tracker,
            trajectory_annotator=traj_annot,
            angle_detector=angle_detector,
            bounce_annotator=bounce_annotator,
            save_csv=True,
            writer=writer,
            conf_threshold=0.25,
            court =  court_coordinates,
            iou_threshold=0.45))
    logging.info('Inference completed')
finally:
    f.close()

    


0: 384x640 (no detections), 32.0ms
Speed: 5.0ms preprocess, 32.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Added court

0: 384x640 (no detections), 27.0ms
Speed: 5.0ms preprocess, 27.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Added court

0: 384x640 (no detections), 28.0ms
Speed: 5.0ms preprocess, 28.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Added court

0: 384x640 (no detections), 28.0ms
Speed: 5.0ms preprocess, 28.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Added court

0: 384x640 (no detections), 25.0ms
Speed: 6.0ms preprocess, 25.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Added court

0: 384x640 (no detections), 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Added court

0: 384x640 (no detections), 16.0ms
Speed: 4.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 64

INFO:root:Inference completed


Added court
